#### Importing external packages

In [1]:
%load_ext autoreload 

%autoreload 2


import numpy as np
import pandas as pd
import sys
from collections import Counter
from keras import layers
import keras
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Resizing
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

#### Importing internal packages

In [2]:
# Add a directory path to sys.path
sys.path.append('/Users/niknmirosh/Code/Yassinoko/dynamic-players-insights/data_processing/')

In [3]:
from get_images import get_players_images_cropped
from class_preprocessing import preprocessing_classes
from image_preprocessing import processing_images

#### Uploading data

In [28]:
data = get_players_images_cropped()
data

{'image': [array([[[ 78,  71,  65],
          [ 38,  31,  25],
          [ 59,  52,  46],
          ...,
          [  9,   8,  14],
          [  9,   8,  14],
          [  9,   8,  14]],
  
         [[ 60,  53,  47],
          [ 71,  64,  58],
          [ 73,  66,  60],
          ...,
          [  9,   8,  14],
          [  9,   8,  14],
          [  8,   7,  13]],
  
         [[ 61,  54,  48],
          [ 82,  75,  69],
          [106,  99,  93],
          ...,
          [  9,   8,  14],
          [  9,   8,  14],
          [  8,   7,  13]],
  
         ...,
  
         [[165,  99,  77],
          [167, 102,  80],
          [166, 104,  83],
          ...,
          [ 13,  13,  15],
          [ 13,  13,  15],
          [ 12,  11,  17]],
  
         [[165,  99,  77],
          [165, 100,  78],
          [165, 103,  82],
          ...,
          [ 13,  13,  15],
          [ 13,  13,  15],
          [ 12,  11,  17]],
  
         [[169, 103,  81],
          [165, 100,  78],
          [163,

In [5]:
plt.imshow(data['image'][0])

IndexError: list index out of range

In [ ]:
# data['name'] = data['name'][:20]
# data['image'] = data['image'][:20]

#### Preprocessing data

In [ ]:
np.mean([s.shape[1] for s in data['image']])

In [44]:
X, y = processing_images(data)
# X, y = data['image'], data['name']

120


100%|█████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 3419.97it/s]

(120, 64, 64, 3)


In [40]:
X.shape

(120, 64, 64, 3)

In [ ]:
plt.imshow(X[5,:,:,:])

In [45]:
y = preprocessing_classes(y)

In [46]:
y.shape

(120, 5)

#### Building augmentation pipeline

In [ ]:
# Compute most common shape to set as an input shape
def most_common(lst):
    counter = Counter(lst)
    most_common_item = max(counter, key=counter.get)
    return most_common_item

shapes = [img.shape for img in data['image']]
common = most_common(shapes)

In [47]:
# img_height = common[0]
# img_width = common[1]

data_augmentation = keras.Sequential(
  [
    layers.RandomFlip(),
    layers.RandomRotation(0.1)
  ]
)


for _ in range(3):
    X = np.concatenate((X, data_augmentation(X)))
    y = np.concatenate((y, y))
    
X.shape, y.shape

((960, 64, 64, 3), (960, 5))

#### Initializing the model

In [62]:
def initialize_model():
    model = models.Sequential()
    # model.add(layers.RandomFlip())
    # model.add(layers.RandomRotation(0.1))
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(y.shape[-1], activation='softmax', name="outputs"))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

#### Splitting the data

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
# ## Loading VGG16 model
# base_model = VGG16(weights="imagenet", include_top=False, input_shape=X_train[0].shape)
# base_model.trainable = False ## Not trainable weights

# ## Preprocessing input
# X_train = preprocess_input(X_train) 
# X_train = preprocess_input(X_test)

In [ ]:
# base_model.summary()

In [ ]:
# flatten_layer = layers.Flatten()
# dense_layer_1 = layers.Dense(50, activation='relu')
# dense_layer_2 = layers.Dense(20, activation='relu')
# prediction_layer = layers.Dense(y.shape[-1], activation='softmax')


# model = models.Sequential([
#     base_model,
#     flatten_layer,
#     dense_layer_1,
#     dense_layer_2,
#     prediction_layer
# ])

In [ ]:
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy'],
# )

In [49]:
X_train.shape

(672, 64, 64, 3)

In [ ]:
plt.imshow(X_train[0,:,:,:])

#### Fitting the model

In [60]:
model = initialize_model()
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 31, 31, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 6, 6, 64)       

In [65]:
model = initialize_model()

es = EarlyStopping(monitor='accuracy', patience=5)

history = model.fit(X_train, y_train, 
                    epochs=50, batch_size=4, 
                    verbose=1, callbacks=[es], 
                    validation_data=(X_test, y_test))

Epoch 1/50
168/168 [==============================] - 1s 7ms/step - loss: 3.7299 - accuracy: 0.3810 - val_loss: 1.3251 - val_accuracy: 0.4653
Epoch 2/50
168/168 [==============================] - 1s 8ms/step - loss: 1.2620 - accuracy: 0.5134 - val_loss: 1.2564 - val_accuracy: 0.5000
Epoch 3/50
168/168 [==============================] - 1s 7ms/step - loss: 1.2223 - accuracy: 0.5193 - val_loss: 1.3108 - val_accuracy: 0.4653
Epoch 4/50
168/168 [==============================] - 1s 7ms/step - loss: 1.0632 - accuracy: 0.5818 - val_loss: 1.2513 - val_accuracy: 0.5382
Epoch 5/50
168/168 [==============================] - 1s 7ms/step - loss: 0.8290 - accuracy: 0.6815 - val_loss: 0.9451 - val_accuracy: 0.6736
Epoch 6/50
168/168 [==============================] - 1s 7ms/step - loss: 0.6701 - accuracy: 0.7560 - val_loss: 1.4001 - val_accuracy: 0.5903
Epoch 7/50
168/168 [==============================] - 1s 7ms/step - loss: 0.7023 - accuracy: 0.7366 - val_loss: 0.8965 - val_accuracy: 0.6493
Epoch 

In [66]:
model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 7ms/step - loss: 1.4147 - accuracy: 0.7812


[1.41470468044281, 0.78125]